In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from IPython.display import display

# Import API key
from api_keys import g_key

In [2]:
gmaps.configure(api_key=g_key)

In [4]:
!jupyter nbextension enable --py --sys-prefix gmaps

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [3]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
cities = pd.read_csv('weatherdata.csv')


In [22]:
cities.head()

,city,temp,lat,lng,humidity,cloudiness,wind_speed,country,date
0,George Town,31.81,5.4112,100.3354,70,20,3.60,MY,1657878205
1,Saint Paul Harbor,13.73,57.7900,-152.4072,82,0,2.57,US,1657878207
2,Aasiaat,3.51,68.7098,-52.8699,79,89,4.15,GL,1657878209
3,São João da Barra,21.05,-21.6403,-41.0511,89,42,4.90,BR,1657878211
4,Hilo,22.82,19.7297,-155.0900,95,100,2.57,US,1657878214


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
#creating a humidity map
locations = cities[["lat",'lng']].astype(float)
humidities = cities["humidity"].astype(float)

# heatmap
fig = gmaps.figure()

#create humidity layer ()
heat_layer = gmaps.heatmap_layer(locations,
    weights=humidities,
    max_intensity=100,
    dissipating=False,
    point_radius=3,
)

#add humidity layer
fig.add_layer(heat_layer) 

#show map with layer
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
cities.query('(temp < 35) and (temp>25) and (cloudiness<20) and (wind_speed<5) and humidity <30')


,city,temp,lat,lng,humidity,cloudiness,wind_speed,country,date
38,Kasongo-Lunda,32.69,-6.4667,16.8167,19,1,2.27,CD,1657878293
154,Deming,25.94,32.2687,-107.7586,27,0,3.09,US,1657878573
181,Beloha,26.77,-25.1667,45.0500,27,0,3.50,MG,1657878637
206,Jalu,30.86,29.0331,21.5482,25,0,3.88,LY,1657878694
252,Malatya Province,29.03,38.5000,38.0000,29,0,2.06,TR,1657878807
305,Timizart,32.44,36.8000,4.2667,29,0,2.83,DZ,1657878927
369,Dinar,26.98,38.0650,30.1656,26,0,2.40,TR,1657879075
379,Lumeje,25.09,-11.5500,20.7833,27,5,4.67,AO,1657879096
382,Nanyamba,30.44,-10.6833,39.8333,29,17,4.61,TZ,1657879102
477,Awjilah,30.75,29.1081,21.2869,25,0,3.68,LY,1657879348


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_df=cities.query('(temp < 35) and (temp>25) and (cloudiness<20) and (wind_speed<5) and humidity <30').reset_index(drop=True)

In [40]:
hotel_df.dropna(how='all')

,city,temp,lat,lng,humidity,cloudiness,wind_speed,country,date
0,Kasongo-Lunda,32.69,-6.4667,16.8167,19,1,2.27,CD,1657878293
1,Deming,25.94,32.2687,-107.7586,27,0,3.09,US,1657878573
2,Beloha,26.77,-25.1667,45.0500,27,0,3.50,MG,1657878637
3,Jalu,30.86,29.0331,21.5482,25,0,3.88,LY,1657878694
4,Malatya Province,29.03,38.5000,38.0000,29,0,2.06,TR,1657878807
5,Timizart,32.44,36.8000,4.2667,29,0,2.83,DZ,1657878927
6,Dinar,26.98,38.0650,30.1656,26,0,2.40,TR,1657879075
7,Lumeje,25.09,-11.5500,20.7833,27,5,4.67,AO,1657879096
8,Nanyamba,30.44,-10.6833,39.8333,29,17,4.61,TZ,1657879102
9,Awjilah,30.75,29.1081,21.2869,25,0,3.68,LY,1657879348


In [44]:
#adding Hotel Name column to the dataframe
hotel_df["Hotel Name"] = ''
hotel_df.head()

,city,temp,lat,lng,humidity,cloudiness,wind_speed,country,date,Hotel Name
0,Kasongo-Lunda,32.69,-6.4667,16.8167,19,1,2.27,CD,1657878293,
1,Deming,25.94,32.2687,-107.7586,27,0,3.09,US,1657878573,
2,Beloha,26.77,-25.1667,45.0500,27,0,3.50,MG,1657878637,
3,Jalu,30.86,29.0331,21.5482,25,0,3.88,LY,1657878694,
4,Malatya Province,29.03,38.5000,38.0000,29,0,2.06,TR,1657878807,


In [47]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params.update({'location': f'{row["lat"]}, {row["lng"]}'})
    response = requests.get(base_url, params=params).json()
    results = response.get('results',[])
    if results:
        hotel_name = results[0]['name'] 
    else:
        hotel_name = 'Hotel not Found'
    hotel_df['Hotel Name'][index] = hotel_name
        
hotel_df.reset_index(drop=True)

/var/folders/pr/15l_881n46j7cwzyn7wprqww0000gp/T/ipykernel_1760/1376817316.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][index] = hotel_name


,city,temp,lat,lng,humidity,cloudiness,wind_speed,country,date,Hotel Name
0,Kasongo-Lunda,32.69,-6.4667,16.8167,19,1,2.27,CD,1657878293,Hotel not Found
1,Deming,25.94,32.2687,-107.7586,27,0,3.09,US,1657878573,Best Western Deming Southwest Inn
2,Beloha,26.77,-25.1667,45.0500,27,0,3.50,MG,1657878637,Hotel not Found
3,Jalu,30.86,29.0331,21.5482,25,0,3.88,LY,1657878694,صاحبة الفخامة
4,Malatya Province,29.03,38.5000,38.0000,29,0,2.06,TR,1657878807,Hotel not Found
5,Timizart,32.44,36.8000,4.2667,29,0,2.83,DZ,1657878927,yefssah restaurant
6,Dinar,26.98,38.0650,30.1656,26,0,2.40,TR,1657879075,Dinar Öğretmenevi
7,Lumeje,25.09,-11.5500,20.7833,27,5,4.67,AO,1657879096,Hotel not Found
8,Nanyamba,30.44,-10.6833,39.8333,29,17,4.61,TZ,1657879102,Busati Lodge Nanyamba
9,Awjilah,30.75,29.1081,21.2869,25,0,3.68,LY,1657879348,المنزل


In [57]:
#dropping columns where hotels where not found
hotel_df=hotel_df[hotel_df["Hotel Name"].str.contains("Hotel not Found")==False]

In [58]:
hotel_df

,city,temp,lat,lng,humidity,cloudiness,wind_speed,country,date,Hotel Name
1,Deming,25.94,32.2687,-107.7586,27,0,3.09,US,1657878573,Best Western Deming Southwest Inn
3,Jalu,30.86,29.0331,21.5482,25,0,3.88,LY,1657878694,صاحبة الفخامة
5,Timizart,32.44,36.8000,4.2667,29,0,2.83,DZ,1657878927,yefssah restaurant
6,Dinar,26.98,38.0650,30.1656,26,0,2.40,TR,1657879075,Dinar Öğretmenevi
8,Nanyamba,30.44,-10.6833,39.8333,29,17,4.61,TZ,1657879102,Busati Lodge Nanyamba
9,Awjilah,30.75,29.1081,21.2869,25,0,3.68,LY,1657879348,المنزل
11,Ampanihy,26.21,-24.7000,44.7500,29,0,3.52,MG,1657879436,Angora Hotel


In [61]:
# NOTE: Do not change any of the code in this cell
# Set parameters to search for hotels with 5000 meters


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [68]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=[f"<br><b>Hotel</b>:</br>{hotel_df['Hotel Name']},\n<br><b>City:</br></b> {hotel_df['city']}, <br><b>Country:</br></b>{hotel_df['country']}" for ind in hotel_df.index])
fig.add_layer(markers)

#Display figure
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

The heat layer clearly shows that cities near the oceans are the ones the highest or are more likely to have more humidity. 
Which would suggests that humidity is influesnced by both the temperatrure and the avalability of larges water
sources such as rivers dams oceans etc. This is clearly more visible in australia and south america continents